In [29]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd

In [2]:
url = 'https://www.unian.ua/politics'

In [ ]:
def get_unian_news_list(url: str):
    req = requests.get(
        url, headers={"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36"})
    if req.status_code != 200:
        return "Status not 200"
    page = BeautifulSoup(req.text, 'html.parser')
    all_news = page.find_all('div', class_='list-thumbs__item')
    return all_news

In [ ]:
def extract_text_from_news(link):
    req = requests.get(link, headers={
                       "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36"})
    if req.status_code != 200:
        return "Status not 200"
    page = BeautifulSoup(req.text, 'html.parser')
    raw_text = page.find('div', class_='article-text')
    try:
        text = '\n'.join([x.getText() for x in raw_text.find_all('p')])
    except:
        return 'Something goes wrong!'
    return text

In [119]:
def ectraxt_data_from_news(news_item):
    print(f"Processing: {news_item.a['href']}")
    news = {'title': '', 'text': '', 'creation_time': '', 'reference': ''}

    news["title"] = news_item.find('h3').getText().strip()
    news["text"] = extract_text_from_news(news_item.a['href'])
    news["creation time"] = news_item.find(
        'div', class_='list-thumbs__time time').getText()
    news['reference'] = news_item.a['href'].strip()
    
    return news

In [ ]:
def gather_news(url):
    gathered_news = []
    try:
        news_urls = get_unian_news_list(url)[:10]
    except:
        return 'Cannot get news list!'

    for item in news_urls:
        gathered_news.append(ectraxt_data_from_news(item))
        time.sleep(3)
        
    return gathered_news

In [114]:
news = gather_news(url)

In [116]:
df = pd.DataFrame.from_dict(news)

In [ ]:
df.to_csv('./test.csv', index=False)